# Reconhecimento de Gesto Dinâmicos Utilizando Redes Neurais Profundas


<br/>**Disciplina**: Introdução a redes neurais profundas 2017/1
<br/>**Professor**: Jorge Leonid Aching Samatelo
<br/>**Alunos**: Clebeson Canuto dos Santos e Leonardo de Assis Silva
    

## Códigos

In [1]:
from __future__ import division, print_function
import tensorflow as tf
import numpy as np
import os

In [2]:
is_test = False

initial_learning_rate = 4e-3
decay_steps = 7e2
decay_rate = 0.98

summary_step = 1e2
batch_size = 256

checkpoint_dir="./checkpoints_gr_20"

n_classes = 20
rows, cols, channels = 128, 128, 3
max_steps = 8e4
input_file = './mixed.tfrecords'
depth = 8192

if is_test:
    max_steps = 20 
    input_file = './test.tfrecords'


In [3]:
def read_decode_distort(queue):
    reader = tf.TFRecordReader()
    _, serialized = reader.read(queue)
    features = tf.parse_single_example(serialized, features={
      'image_raw': tf.FixedLenFeature([rows*cols*channels], tf.string),
      'label': tf.FixedLenFeature([], tf.int64),
    })
    
    label = tf.cast(features['label'], tf.int32)

    image = tf.decode_raw(features['image_raw'], tf.uint8)
    image = tf.cast(image, tf.float32) 
    image = tf.reshape(image, [rows, cols, channels])
    #image = tf.image.rgb_to_hsv(image) # HSV image

    
    #image = tf.image.resize_image_with_crop_or_pad(image, 96, 96)
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_brightness(image, max_delta=63)
    #image = tf.image.random_contrast(image, lower=0.2, upper=1.8)
    
    image = tf.image.per_image_standardization(image)
    return image, label


def load_inputs(filename, batch_size):
    with tf.name_scope('input'):
        queue = tf.train.string_input_producer([filename])
        
        image, label = read_decode_distort(queue)
        
        image_batch, label_batch = tf.train.shuffle_batch(
            [image, label], 
            batch_size = batch_size, 
            num_threads = 4, 
            capacity = 1000 + 3*batch_size, 
            min_after_dequeue = 1000)
    
    #tf.summary.image('images', image_batch)
    label_batch = tf.one_hot(label_batch, n_classes)
    return image_batch, label_batch

In [4]:
with tf.device('/cpu:0'):
    images, labels = load_inputs(input_file, batch_size)
    vimages,vlabels= load_inputs('./validation.tfrecords', batch_size)

keep = tf.placeholder(tf.float32)

global_step = tf.contrib.framework.get_or_create_global_step()

learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, 
                                           decay_steps, decay_rate, staircase=True)
tf.summary.scalar("learning_rate", learning_rate)

tensor_name = "pool5" #"conv3_3/Relu"

with open("vgg16-20160129.tfmodel", mode='rb') as f:
    content = f.read()
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(content)
    
#     graph_def = tf.graph_util.extract_sub_graph(graph_def, ["images", tensor_name])
    graph_def = tf.graph_util.extract_sub_graph(graph_def, ["images", "pool5"])
    tf.import_graph_def(graph_def, input_map={"images": images})
    
del content
graph = tf.get_default_graph()

# net = graph.get_tensor_by_name("import/{0}:0".format(tensor_name))
net = graph.get_tensor_by_name("import/pool5:0")

out_shape = net.get_shape().as_list()
#net = tf.stop_gradient(net)

In [5]:
 graph.get_operations()


[<tf.Operation 'input/input_producer/Const' type=Const>,
 <tf.Operation 'input/input_producer/Size' type=Const>,
 <tf.Operation 'input/input_producer/Greater/y' type=Const>,
 <tf.Operation 'input/input_producer/Greater' type=Greater>,
 <tf.Operation 'input/input_producer/Assert/Const' type=Const>,
 <tf.Operation 'input/input_producer/Assert/Assert/data_0' type=Const>,
 <tf.Operation 'input/input_producer/Assert/Assert' type=Assert>,
 <tf.Operation 'input/input_producer/Identity' type=Identity>,
 <tf.Operation 'input/input_producer/RandomShuffle' type=RandomShuffle>,
 <tf.Operation 'input/input_producer' type=FIFOQueueV2>,
 <tf.Operation 'input/input_producer/input_producer_EnqueueMany' type=QueueEnqueueManyV2>,
 <tf.Operation 'input/input_producer/input_producer_Close' type=QueueCloseV2>,
 <tf.Operation 'input/input_producer/input_producer_Close_1' type=QueueCloseV2>,
 <tf.Operation 'input/input_producer/input_producer_Size' type=QueueSizeV2>,
 <tf.Operation 'input/input_producer/Cast'

In [ ]:
weights = {
    "Wfc1": tf.Variable(tf.truncated_normal([np.prod(out_shape[1:]), depth]), name="fc1/weights"), 
    "Wfc2": tf.Variable(tf.truncated_normal([depth, depth]), name="fc2/weights"), 
    "Wfc3": tf.Variable(tf.truncated_normal([depth, n_classes]), name="fc3/weights")
} 

biases = {
    "Bfc1": tf.Variable(tf.truncated_normal([depth]), name="fc1/biases"),
    "Bfc2": tf.Variable(tf.truncated_normal([depth]), name="fc2/biases"),
    "Bfc3": tf.Variable(tf.truncated_normal([n_classes]), name="fc3/biases")
}

def inference(net, weigths, biases, keep):
    net = tf.reshape(net, [-1, weights['Wfc1'].get_shape().as_list()[0]], name="fc1/reshape")
    net = tf.add(tf.matmul(net, weights['Wfc1']), biases['Bfc1'], name="fc1/matmul-add")
    net = tf.nn.relu(net, name="fc1/relu")
    net = tf.nn.dropout(net, keep, name="fc1/dropout")

    net = tf.add(tf.matmul(net, weights['Wfc2']), biases['Bfc2'], name="fc2/matmul-add")
    net = tf.nn.relu(net, name="fc2/relu")
    net = tf.nn.dropout(net, keep, name="fc2/dropout")

    net = tf.add(tf.matmul(net, weights['Wfc3']), biases['Bfc3'], name="fc3/matmul-add")
    return net

with tf.name_scope('model'):
    pred = inference(net, weights, biases, keep)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=labels))
    tf.summary.scalar("loss", loss)

with tf.name_scope('sgd'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

with tf.name_scope('accuracy'):
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(labels, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
    tf.summary.scalar("accuracy", acc)

In [ ]:

def formatter(d): 
    return 'loss: {0}, acc: {1}%'.format(d.values()[0], d.values()[1]*100)

# with tf.device('/gpu:0'):
hooks = [
    tf.train.StopAtStepHook(num_steps=max_steps),
    tf.train.LoggingTensorHook([loss, acc], every_n_secs=30, formatter=formatter),
    tf.train.NanTensorHook(loss),
]

with tf.train.MonitoredTrainingSession(checkpoint_dir=checkpoint_dir,
                                       hooks=hooks,
                                       save_checkpoint_secs=10*60,
                                       save_summaries_steps=summary_step) as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    try:
        if not is_test:
            while not coord.should_stop() or sess.should_stop():
                
                    _, step = sess.run([optimizer, global_step], feed_dict={keep: 0.5})
                    
#                     if step % 10 == 0:
#                         sum_accurace = 0
#                         for i in range(20):
#                             test_accuracy, step = sess.run([acc, global_step],  feed_dict={images:vimages, labels:vlabels, keep: 1.0})
#                             sum_accurace += test_accuracy * batch_size

#                         print("Validation Accuracy: %2f" % (sum_accurace / (batch_size * count)))
                                
        else:
            sum_accurace = 0
            count = 0
            while not coord.should_stop() or sess.should_stop():
    #             _, step = sess.run([optimizer, global_step], feed_dict={keep: 0.5})

                    test_accuracy, step = sess.run([acc, global_step],  feed_dict={keep: 1.0})
                    #print("Test Accuracy: %2f" % test_accuracy , "global_step: %d" % int(step//1))
                    count += 1
                    sum_accurace += test_accuracy*batch_size
                    if count == max_steps:
                        break;
            print("Test Accuracy: %2f" % (sum_accurace / (batch_size * count)))

    except tf.errors.OutOfRangeError:
        print('Done training...')
    finally:
        coord.request_stop()

    coord.join(threads)
    #sess.close()

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./checkpoints_gr_20/model.ckpt.
INFO:tensorflow:loss: 106686600.0, acc: 5.078125%
INFO:tensorflow:loss: 22070288.0, acc: 34.765625%
INFO:tensorflow:global_step/sec: 1.81731
INFO:tensorflow:loss: 11700713.0, acc: 39.0625%
INFO:tensorflow:loss: 8393341.0, acc: 41.796875%
INFO:tensorflow:global_step/sec: 1.89835
INFO:tensorflow:loss: 5279816.0, acc: 50.0%
INFO:tensorflow:loss: 4390669.0, acc: 51.171875%
INFO:tensorflow:global_step/sec: 1.89147
INFO:tensorflow:loss: 3945717.0, acc: 51.5625%
INFO:tensorflow:global_step/sec: 1.89091
INFO:tensorflow:loss: 3318072.5, acc: 55.46875%
INFO:tensorflow:loss: 3063143.0, acc: 52.734375%
INFO:tensorflow:global_step/sec: 1.88953
INFO:tensorflow:loss: 2834282.5, acc: 57.03125%
INFO:tensorflow:loss: 2264813.25, acc: 60.546875%
INFO:tensorflow:global_step/sec: 1.89229
INFO:tensorflow:loss: 2091689.25, acc: 61.71875%
INFO:tensorflow:loss: 2057404.875, acc: 61.328125%


INFO:tensorflow:loss: 236093.28125, acc: 91.015625%
INFO:tensorflow:global_step/sec: 1.89077
INFO:tensorflow:loss: 197973.375, acc: 92.96875%
INFO:tensorflow:loss: 154785.203125, acc: 94.140625%
INFO:tensorflow:global_step/sec: 1.88979
INFO:tensorflow:loss: 133703.0, acc: 94.53125%
INFO:tensorflow:global_step/sec: 1.89014
INFO:tensorflow:loss: 154751.796875, acc: 94.140625%
INFO:tensorflow:loss: 342845.5, acc: 92.578125%
INFO:tensorflow:global_step/sec: 1.88976
INFO:tensorflow:loss: 131262.15625, acc: 94.921875%
INFO:tensorflow:loss: 93385.421875, acc: 95.3125%
INFO:tensorflow:global_step/sec: 1.8902
INFO:tensorflow:loss: 150651.296875, acc: 95.3125%
INFO:tensorflow:loss: 79829.015625, acc: 96.875%
INFO:tensorflow:global_step/sec: 1.89075
INFO:tensorflow:loss: 61357.7382812, acc: 96.484375%
INFO:tensorflow:global_step/sec: 1.8909
INFO:tensorflow:loss: 116815.539062, acc: 95.703125%
INFO:tensorflow:loss: 96132.640625, acc: 94.140625%
INFO:tensorflow:Saving checkpoints for 6788 into ./ch

INFO:tensorflow:global_step/sec: 1.8887
INFO:tensorflow:loss: 16343.3300781, acc: 98.828125%
INFO:tensorflow:global_step/sec: 1.88915
INFO:tensorflow:loss: 41991.1328125, acc: 97.265625%
INFO:tensorflow:loss: 26178.4941406, acc: 98.046875%
INFO:tensorflow:global_step/sec: 1.88784
INFO:tensorflow:loss: 67686.875, acc: 96.875%
INFO:tensorflow:loss: 28371.1523438, acc: 98.046875%
INFO:tensorflow:global_step/sec: 1.88992
INFO:tensorflow:loss: 91993.671875, acc: 96.875%
INFO:tensorflow:Saving checkpoints for 12444 into ./checkpoints_gr_20/model.ckpt.
INFO:tensorflow:loss: 145608.625, acc: 94.53125%
INFO:tensorflow:global_step/sec: 1.82587
INFO:tensorflow:loss: 32084.6210938, acc: 97.65625%
INFO:tensorflow:global_step/sec: 1.888
INFO:tensorflow:loss: 24900.25, acc: 98.828125%
INFO:tensorflow:loss: 89716.53125, acc: 97.265625%
INFO:tensorflow:global_step/sec: 1.8865
INFO:tensorflow:loss: 7015.2734375, acc: 99.609375%
INFO:tensorflow:loss: 38926.5234375, acc: 97.65625%
INFO:tensorflow:global_s

In [ ]:
#exp 9  (rgb, 256, stop, train, 4096, e-3)
#exp 10 (rgb, 256, stop, mixed, 4096, e-3)
#exp 11 (rgb, 256, fine, mixed, 4096, e-3)
#exp 12 (rgb, 128, stop, mixed, 4096, e-3)
#exp 13 (rgb, 128, stop, mixed, 8192, e-3) 75%
#exp 14 (rgb, 128, fine, mixed, 8192, e-3) 73%
#exp 15 (rgb, 128, fine, mixed, 4096, e-3) 33%
#exp 16 (rgb, 256, fine, mixed, 8192, e-3) 75%
#exp 17 (hsv, 256, fine, mixed, 8192, e-3) 70%
#exp 18 (rgb, 64, fine, mixed, 8192,  e-2) 26%
#exp 19 (rgb, 256, fine, mixed, 8192,  4e-2) 5%
#exp 20 (rgb, 256, fine, mixed, 8192,  4e-3) 5%

## 